<a href="https://colab.research.google.com/github/Mohann23/PySpark_Material_Practice/blob/master/rough.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Rough notes

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!apt-get install openjdk-8-jdk-headless -qq > /dev/null


!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz



!tar xf spark-3.2.1-bin-hadoop3.2.tgz

!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

import findspark
findspark.init()

findspark.find()
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

Mounted at /content/drive


In [2]:
spark = SparkSession.builder.appName('Task_6').getOrCreate()

In [3]:
user_log_df = spark.read.csv('/content/drive/MyDrive/PYSpark_Files/user_login_history_Task_6.csv',inferSchema=True,header=True)

In [4]:
user_log_df.show(5)

+---+------+----------+
| id|userid|login_date|
+---+------+----------+
|  1|     1|01-01-2019|
|  2|     2|02-01-2019|
|  3|     3|03-01-2019|
|  4|     4|04-01-2019|
|  5|     5|05-01-2019|
+---+------+----------+
only showing top 5 rows



In [5]:
user = user_log_df.groupBy('userid','login_date').agg(count('userid').alias('current_year'))

In [6]:
user.show()

+------+----------+------------+
|userid|login_date|current_year|
+------+----------+------------+
|    12|30-07-2020|           1|
|     6|25-02-2021|           1|
|     6|25-07-2021|           1|
|    13|15-04-2022|           1|
|    26|26-01-2020|           1|
|     3|01-06-2020|           1|
|    43|12-02-2021|           1|
|    44|04-04-2021|           1|
|    24|12-08-2021|           1|
|     9|05-11-2021|           1|
|     1|01-02-2019|           1|
|     4|22-07-2020|           1|
|    11|29-07-2020|           1|
|    20|11-03-2021|           1|
|    13|23-04-2021|           1|
|    18|28-04-2021|           1|
|     8|08-03-2019|           1|
|    19|09-03-2020|           1|
|    14|01-08-2020|           1|
|     7|26-07-2021|           1|
+------+----------+------------+
only showing top 20 rows



In [7]:
from datetime import datetime

def try_parsing_date(text):
    for fmt in ('%Y-%m-%d', '%d.%m.%Y', '%d/%m/%Y','%d-%m-%Y','%dd-%mm-%yyyy'):
        try:
            date_object = datetime.strptime(text, fmt)
            converted_date_object = date_object.strftime('%Y')
            return converted_date_object

        except ValueError:
            pass
    return 'ERROR_DATE'

def try_parsing_date_m(text):
    for fmt in ('%Y-%m-%d', '%d.%m.%Y', '%d/%m/%Y','%d-%m-%Y','%dd-%mm-%yyyy'):
        try:
            date_object = datetime.strptime(text, fmt)
            converted_date_object = date_object.strftime('%m')
            return converted_date_object

        except ValueError:
            pass
    return 'ERROR_DATE'


In [8]:
year_handler = udf(lambda x : try_parsing_date(x))
month_handler = udf(lambda x : try_parsing_date_m(x))

In [9]:
df = user_log_df.withColumn('Current_year',year_handler(col('login_date')))
df = df.withColumn('Current_Month',month_handler(col('login_date')))

In [10]:
df.show()

+---+------+----------+------------+-------------+
| id|userid|login_date|Current_year|Current_Month|
+---+------+----------+------------+-------------+
|  1|     1|01-01-2019|        2019|           01|
|  2|     2|02-01-2019|        2019|           01|
|  3|     3|03-01-2019|        2019|           01|
|  4|     4|04-01-2019|        2019|           01|
|  5|     5|05-01-2019|        2019|           01|
|  6|     6|06-01-2019|        2019|           01|
|  7|     7|07-01-2019|        2019|           01|
|  8|     8|08-01-2019|        2019|           01|
|  9|    68|08-01-2019|        2019|           01|
| 10|     9|09-01-2019|        2019|           01|
| 11|    10|10-01-2019|        2019|           01|
| 12|     1|01-02-2019|        2019|           02|
| 13|     2|02-02-2019|        2019|           02|
| 14|     3|03-02-2019|        2019|           02|
| 15|     4|04-02-2019|        2019|           02|
| 16|     5|05-02-2019|        2019|           02|
| 17|     6|06-02-2019|        

In [11]:
user = df.groupBy('userid','login_date').agg(count('userid').alias('current_year'),(count('userid').alias('current_month')))

In [26]:
user_2 = user.select(countDistinct('userid','Current_Month').alias('mmmm'))

In [27]:
user_2.show()

+----+
|mmmm|
+----+
|  51|
+----+

